## 문자열 시리즈 데이터 평가와 정제
- 문자열 평가와 조작을 위한 도구를 폭넓게 가지는 것이 중요하다.

In [1]:
import pandas as pd
import numpy as np

nls97=pd.read_csv('C:/data-cleansing-main/Chapter06/data/nls97c.csv')
nls97.set_index('personid',inplace=True)

### 문자열에 패턴이 존재하는지 검사

In [2]:
nls97.govprovidejobs.value_counts()

2. Probably          617
3. Probably not      462
1. Definitely        454
4. Definitely not    300
Name: govprovidejobs, dtype: int64

In [6]:
nls97['govprovidejobsdefprob']=np.where(nls97.govprovidejobs.isnull(),
                                        np.nan, np.where(nls97.govprovidejobs.str.contains('not'),'No','Yes'))      # 응답에 not이 포함되는지 contains로 조사
                                        # where을 호출할 때 누락값을 먼저 처리해서, 첫 번째 else절에서 끝나버리지 않게 함(?)
pd.crosstab(nls97.govprovidejobs,nls97.govprovidejobsdefprob)

govprovidejobsdefprob,No,Yes
govprovidejobs,,
1. Definitely,0,454
2. Probably,0,617
3. Probably not,462,0
4. Definitely not,300,0


### 문자열의 시작, 끝에 있는 공백 처리

In [8]:
nls97.maritalstatus.value_counts()

Married          3064
Never-married    2766
Divorced          663
Separated         154
Widowed            23
Married             2
Name: maritalstatus, dtype: int64

In [11]:
nls97.maritalstatus.str.startswith(' ').any()       # 앞에 공백이 있는지?

False

In [12]:
nls97.maritalstatus.str.endswith(' ').any()         # 뒤에 공백이 있는지?

True

In [14]:
nls97['evermarried']=np.where(nls97.maritalstatus.isnull(),         # np.where에서 두번째, 세번째 인자에 True일 때 값, False일 때 값을 써주면 브로드 캐스팅이 가능
                              np.nan, np.where(nls97.maritalstatus.str.strip()=='Never-married','No','Yes')) 
pd.crosstab(nls97.maritalstatus,nls97.evermarried)

evermarried,No,Yes
maritalstatus,,
Divorced,0,663
Married,0,3064
Married,0,2
Never-married,2766,0
Separated,0,154
Widowed,0,23


### isin을 이용해 문자열 값이 리스트에 있는지 검사

In [15]:
nls97['receivedba']=np.where(nls97.highestdegree.isnull(),
                             np.nan, np.where(nls97.highestdegree.str[0:1].isin(['4','5','6','7']),'Yes','No'))

pd.crosstab(nls97.highestdegree,nls97.receivedba)

receivedba,No,Yes
highestdegree,,
0. None,953,0
1. GED,1146,0
2. High School,3667,0
3. Associates,737,0
4. Bachelors,0,1673
5. Masters,0,603
6. PhD,0,54
7. Professional,0,120


### findall을 이용해 문자열에서 숫자값 추출
- finditer()는 정규식과 매치되는 모든 문자열을 iterator 객체로 리턴한다.
- iterator 객체의 값을 불러오려면 for문을 이용해 읽어들여야 한다.
- findall()은 정규식과 매치되는 모든 문자열을 리스트형식으로 리턴한다.

In [16]:
pd.concat([nls97.weeklyhrstv.head(),
           nls97.weeklyhrstv.str.findall('\d+').head()],axis=1)     # findall을 사용해 문자열에 포함된 모든 숫자의 리스트를 생성
                                                                    # 문자열에 포함된 숫자만 얻어내기 위해 findall에 정규표현식 '\d+'를 전달(블로그 저장)

,weeklyhrstv,weeklyhrstv
personid,,
100061,11 to 20 hours a week,"[11, 20]"
100139,3 to 10 hours a week,"[3, 10]"
100284,11 to 20 hours a week,"[11, 20]"
100292,NaN,NaN
100583,3 to 10 hours a week,"[3, 10]"


In [18]:
# findall로 만든 리스트를 사용해 weeklyhrstv 텍스트로부터 숫자 시리즈를 생성
def getnum(numlist):                # 각각의 weeklyhrstv에 대해 findall로 만들어낸 리스트에서 마지막 원소를 검색하는 함수 정의
  highval = 0
  if (type(numlist) is list):
    lastval = int(numlist[-1])
    if (numlist[0]=='40'):
      highval = 45
    elif (lastval==2):
      highval = 1
    else:
      highval = lastval - 5
  else:
    highval = np.nan
  return highval

In [19]:
nls97['weeklyhrstvnum']=nls97.weeklyhrstv.str.findall('\d+').apply(getnum)
pd.crosstab(nls97.weeklyhrstv,nls97.weeklyhrstvnum)

weeklyhrstvnum,1.0,5.0,15.0,25.0,35.0,45.0
weeklyhrstv,,,,,,
11 to 20 hours a week,0,0,1145,0,0,0
21 to 30 hours a week,0,0,0,299,0,0
3 to 10 hours a week,0,3625,0,0,0,0
31 to 40 hours a week,0,0,0,0,116,0
Less than 2 hours per week,1350,0,0,0,0,0
More than 40 hours a week,0,0,0,0,0,176


### 시리즈에 있는 값 바꾸기

In [21]:
comphrsold = ['None','Less than 1 hour a week','1 to 3 hours a week','4 to 6 hours a week','7 to 9 hours a week','10 hours or more a week']
comphrsnew = ['A. None','B. Less than 1 hour a week','C. 1 to 3 hours a week','D. 4 to 6 hours a week','E. 7 to 9 hours a week','F. 10 hours or more a week']

nls97.weeklyhrscomputer.value_counts().sort_index()      # 정렬이 잘 안됨

1 to 3 hours a week         733
10 hours or more a week    3669
4 to 6 hours a week         726
7 to 9 hours a week         368
Less than 1 hour a week     296
None                        918
Name: weeklyhrscomputer, dtype: int64

In [22]:
nls97.weeklyhrscomputer.replace(comphrsold,comphrsnew,inplace=True)
nls97.weeklyhrscomputer.value_counts().sort_index()     # 정렬 되도록 값을 바꿈

A. None                        918
B. Less than 1 hour a week     296
C. 1 to 3 hours a week         733
D. 4 to 6 hours a week         726
E. 7 to 9 hours a week         368
F. 10 hours or more a week    3669
Name: weeklyhrscomputer, dtype: int64

## 정리
1. contains()
2. isin()
3. findall + 정규표현식